<a href="https://colab.research.google.com/github/picazio/HUGWork/blob/master/DataCleaning_FillEmptyFieldsWithPatientZeroData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Import all libraries

In [0]:
import pandas as pd
import numpy as np
from datetime import datetime,timedelta
from sklearn.metrics import mean_squared_error
from scipy.optimize import curve_fit
from scipy.optimize import fsolve
import matplotlib.pyplot as plt
%matplotlib inline

Import dataframe and explore it

In [0]:
url="/Users/euclide/Desktop/update_data_29_04.xls"
df = pd.read_excel(io=url, sheet_name='Sheet1')

df.head()

,Codbar,age,date_birth,sex,rue1,ville,nip,pays,date_visit,IgA_Ratio,IgA_Résultat,IgG_Ratio,IgG_Résultat,date_lab
0,12108131,59.0,1961-08-01,m,NaN,NaN,NaN,NaN,2020-04-03,0.221,NEG,0.202,NEG,2020-04-06
1,12108133,19.0,2001-02-01,m,NaN,NaN,NaN,NaN,2020-04-03,0.731,NEG,0.290,NEG,2020-04-06
2,41859912,24.0,1996-04-16,f,CHEMIN DU SAUT DU LOUP 1,CHENE BOURG,1225.0,NaN,2020-04-03,1.282,POS,0.231,NEG,2020-04-06
3,12108941,31.0,1989-01-16,f,NaN,NaN,NaN,NaN,NaT,0.416,NEG,0.166,NEG,2020-04-06
4,1610159,50.0,1969-10-26,m,CHEMIN DE L'EGLISE 5,BERNEX,1233.0,CH,2020-04-03,0.547,NEG,0.207,NEG,2020-04-06


Make the upper case of the address



In [0]:
#df['rue1_correct']=df['rue1'].apply(lambda x: str(x).upper())
df['rue1']=df['rue1'].apply(lambda x: str(x).upper())

Shift house number to the end of the address

In [0]:
def shift_hnumber(address) :
    tokens=address.split(' ')
    hn=''
    indexhn=0
    newAddress=''
    ntokens=0
    for itok,tok in enumerate(tokens) :
        if any(map(str.isdigit, tok)) :
            hn=tok
            indexhn=itok
    if len(tokens)-1!=indexhn :
        if len(tokens[indexhn+1])==1 or tokens[indexhn+1]=='BIS' :
            hn=hn+tokens[indexhn+1]
            ntokens=2
    for itok,tok in enumerate(tokens):
        if (itok==indexhn) or (ntokens==2 and itok==indexhn+1) :
            continue
        newAddress=newAddress+tok+' '
    return newAddress+hn

Remove any commas from the address

In [0]:
#df['rue1_correct']=df['rue1'].apply(lambda x: ''.join(str(x).split(',')))
df['rue1']=df['rue1'].apply(lambda x: ''.join(str(x).split(',')))

Swap the position of the house number at the end of the address

In [0]:
#df['rue1_correct']=df['rue1_correct'].apply(lambda x: shift_hnumber(str(x)))
df['rue1']=df['rue1'].apply(lambda x: shift_hnumber(str(x)))

Change 'AV' in 'AVENUE'

In [0]:
df['rue1']=df['rue1'].apply(lambda x: 'AVENUE'.join(str(x).split('AV ')) if str(x).split(' ')[0]=='AV' else str(x))

In [0]:
df['rue1'].tail()

1403       CHEMIN DES VERJUS 113A
1404          RUE DE L ATHENEE 40
1405    ROUTE DU MOULIN ROGET 43A
1406           RUE DE CANDOLLE 18
1407            RUE ROTHSCHILD 27
Name: rue1, dtype: object

Now let's add the household_id column

In [0]:
def create_households(x) :
    listold=['11','21','31']
    listold2=['410','411']
    if len(str(x))==6 :
        return str(x)
    elif (any(c in str(x)[:2] for c in listold) and len(str(x))==7) :
        return str(x)[:-1]
    elif len(str(x))==8 :
        return str(x)[:-1]
    elif (any(c in str(x)[:3] for c in listold2) and len(str(x))==7) :
        return str(x)[:-1]
    else :
        return str(x)

In [0]:
df['Household_id']=df['Codbar'].apply(lambda x: create_households(x))
df['nr_personnes']=df['Household_id'].map(df['Household_id'].value_counts())

Inspect the result

In [0]:
df.head()

,Codbar,age,date_birth,sex,rue1,ville,nip,pays,date_visit,IgA_Ratio,IgA_Résultat,IgG_Ratio,IgG_Résultat,date_lab,Household_id,nr_personnes
0,12108131,59.0,1961-08-01,m,,NaN,NaN,NaN,2020-04-03,0.221,NEG,0.202,NEG,2020-04-06,1210813,4
1,12108133,19.0,2001-02-01,m,,NaN,NaN,NaN,2020-04-03,0.731,NEG,0.290,NEG,2020-04-06,1210813,4
2,41859912,24.0,1996-04-16,f,CHEMIN DU SAUT DU LOUP 1,CHENE BOURG,1225.0,NaN,2020-04-03,1.282,POS,0.231,NEG,2020-04-06,4185991,3
3,12108941,31.0,1989-01-16,f,,NaN,NaN,NaN,NaT,0.416,NEG,0.166,NEG,2020-04-06,1210894,2
4,1610159,50.0,1969-10-26,m,CHEMIN DE L'EGLISE 5,BERNEX,1233.0,CH,2020-04-03,0.547,NEG,0.207,NEG,2020-04-06,1610159,5


In [0]:
df.isnull().sum()

Codbar           0
age              2
date_birth       2
sex              2
rue1             0
ville           30
nip             32
pays            59
date_visit      72
IgA_Ratio        0
IgA_Résultat     0
IgG_Ratio        0
IgG_Résultat     0
date_lab         0
Household_id     0
nr_personnes     0
dtype: int64

In [0]:
#this to accunt for missing values that are not NaN
print(df[df['rue1']==''].shape[0])

0


The following code is going to work only with a correct ordering of the df

In [0]:
df['support_codbar']=df.apply(lambda x: str(x.Codbar)+'0' if str(x.Codbar)==str(x.Household_id) else str(x.Codbar), axis=1)
df=df.sort_values(by=['support_codbar'])
df.head()

To make the task, we need to create some supporting columns

In [0]:
def fillEmptyFields(listColumns,df) :
    temp=''
    suppDic={}
    for col in listColumns :
        suppDic[col]={'list':[],'suppVar':''}
        
        for index, row in df.iterrows():
            temp=row['Household_id']
            if str(row['Codbar'])==str(temp) :

                suppDic[col]['suppVar']=row[col]
                suppDic[col]['list'].append(suppDic[col]['suppVar'])

            elif str(temp)==str(row['Codbar'])[:-1] :
                suppDic[col]['list'].append(suppDic[col]['suppVar'])
                

        df['Household_'+col]=suppDic[col]['list']

In [0]:
listColumns=[
    'rue1',
    'ville',
    'nip',
    'pays',
    'date_visit'
]
fillEmptyFields(listColumns,df)

Now we can fill the empty of NaN cells with the one of patient0

In [0]:
df['rue1']       =df.apply(lambda x: x.Household_rue1       if pd.isna(x.rue1)        or x.rue1==''      else x.rue1, axis=1)
df['ville']      =df.apply(lambda x: x.Household_ville      if pd.isna(x.ville)       or x.ville==''     else x.ville, axis=1)
df['nip']        =df.apply(lambda x: x.Household_nip        if pd.isna(x.nip)         or str(x.nip)==''  else x.nip, axis=1)
df['pays']       =df.apply(lambda x: x.Household_pays       if pd.isna(x.pays)        or x.pays==''      else x.pays, axis=1)
df['date_visit'] =df.apply(lambda x: x.Household_date_visit if pd.isna(x.date_visit)  or str(x.pays)=='' else x.date_visit, axis=1)


We can clean up the df

In [0]:
df=df.drop(columns=['Household_rue1', 'Household_ville','Household_nip','Household_pays','Household_date_visit','support_codbar'])

Write the Dates in proper format

In [0]:
dates=['date_birth','date_visit','date_lab']
for date in dates :
    df[date] = pd.to_datetime(df[date]).dt.date

Remove wrong entries with undetermined age

In [0]:
df=df[df['age'].notnull()]

In [0]:
df.isnull().sum()

Codbar           0
age              0
date_birth       0
sex              0
rue1             0
ville            3
nip              3
pays             7
date_visit      20
IgA_Ratio        0
IgA_Résultat     0
IgG_Ratio        0
IgG_Résultat     0
date_lab         0
Household_id     0
nr_personnes     0
dtype: int64

In [0]:
df.to_excel("/Users/euclide/Desktop/update_data_29_04a.xlsx")  